In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/WV_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4473367300,WV,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,5.402595e+14
1,5936297000,WV,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,5.403303e+14
2,3467218209,WV,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,5.408596e+14
3,2180937106,WV,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,5.407902e+14
4,5506907200,WV,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,5.405396e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4473367300,WV,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,5.402595e+14
1,5936297000,WV,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,5.403303e+14
2,3467218209,WV,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,5.408596e+14
3,2180937106,WV,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,5.407902e+14
4,5506907200,WV,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,5.405396e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
33,1092927109,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1076,9253917006,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1246,7284168605,WV,"5450 US Route 60, Huntington, WV",37.068304,-88.705767,NaN
1377,6167388303,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
1378,5082197000,WV,"2334 Route 52, KENOVA, WV",35.396151,-80.199428,NaN
2010,7369837107,WV,"1020 McDermott Ridge Road, Buckhannon, WV",38.972506,-80.208209,NaN
2472,6046968304,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
3164,5550167102,WV,"147 11TH AVE ste 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
3186,1014389001,WV,"835 Oakdale Dr, Princeton, WV",33.776886,-84.338984,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('54')] #WV
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/9 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

9


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13876/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
33,1092927109,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,[]
1076,9253917006,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,[]
1246,7284168605,WV,"5450 US Route 60, Huntington, WV",37.068304,-88.705767,[]
1377,6167388303,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,[]
1378,5082197000,WV,"2334 Route 52, KENOVA, WV",35.396151,-80.199428,[]
2010,7369837107,WV,"1020 McDermott Ridge Road, Buckhannon, WV",38.972506,-80.208209,[540979671002039]
2472,6046968304,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,[]
3164,5550167102,WV,"147 11TH AVE ste 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,[]
3186,1014389001,WV,"835 Oakdale Dr, Princeton, WV",33.776886,-84.338984,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13876/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
33,1092927109,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1076,9253917006,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1246,7284168605,WV,"5450 US Route 60, Huntington, WV",37.068304,-88.705767,NaN
1377,6167388303,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
1378,5082197000,WV,"2334 Route 52, KENOVA, WV",35.396151,-80.199428,NaN
2010,7369837107,WV,"1020 McDermott Ridge Road, Buckhannon, WV",38.972506,-80.208209,540979671002039
2472,6046968304,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
3164,5550167102,WV,"147 11TH AVE ste 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
3186,1014389001,WV,"835 Oakdale Dr, Princeton, WV",33.776886,-84.338984,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
33,1092927109,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1076,9253917006,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1246,7284168605,WV,"5450 US Route 60, Huntington, WV",37.068304,-88.705767,NaN
1377,6167388303,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
1378,5082197000,WV,"2334 Route 52, KENOVA, WV",35.396151,-80.199428,NaN
2472,6046968304,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
3164,5550167102,WV,"147 11TH AVE ste 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
3186,1014389001,WV,"835 Oakdale Dr, Princeton, WV",33.776886,-84.338984,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
33,1092927109,NaN
1076,9253917006,NaN
1246,7284168605,NaN
1377,6167388303,NaN
1378,5082197000,NaN
2010,7369837107,540979671002039
2472,6046968304,NaN
3164,5550167102,NaN
3186,1014389001,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/WV_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,960042,4473367300,WV,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,5.402595e+14
1,960043,5936297000,WV,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,5.403303e+14
2,960044,3467218209,WV,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,5.408596e+14
3,960045,2180937106,WV,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,5.407902e+14
4,960046,5506907200,WV,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,5.405396e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,960042,4473367300,WV,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,5.402595e+14,NaN
1,960043,5936297000,WV,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,5.403303e+14,NaN
2,960044,3467218209,WV,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,5.408596e+14,NaN
3,960045,2180937106,WV,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,5.407902e+14,NaN
4,960046,5506907200,WV,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,5.405396e+14,NaN
...,...,...,...,...,...,...,...,...
3360,963402,7867607002,WV,"903 Grand Central Ave, VIENNA, WV",39.308604,-81.553172,5.410701e+14,NaN
3361,963403,8194978504,WV,"218 D St, South Charleston, WV",38.365271,-81.693659,5.403901e+14,NaN
3362,963404,9443798505,WV,"917 Deer Ridge Road, Davis, WV",39.052267,-79.410183,5.409397e+14,NaN
3363,963405,9755898608,WV,"818 W Main St, Bridgeport, WV",39.282200,-80.266586,5.403303e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,960042,4473367300,WV,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,5.402595e+14,NaN,540259501012016.0
1,960043,5936297000,WV,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,5.403303e+14,NaN,540330320002080.0
2,960044,3467218209,WV,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,5.408596e+14,NaN,540859623003039.0
3,960045,2180937106,WV,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,5.407902e+14,NaN,540790206091045.0
4,960046,5506907200,WV,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,5.405396e+14,NaN,540539550004006.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4473367300,WV,"101 Main Street, West, WHITE SULPHUR SPRINGS, WV",37.791294,-80.302276,540259501012016.0
1,5936297000,WV,"600 White Oaks Blvd, BRIDGEPORT, WV",39.335126,-80.230399,540330320002080.0
2,3467218209,WV,"1137 E. Washington Ave, ELLENBORO, WV",39.276860,-81.043076,540859623003039.0
3,2180937106,WV,"1944 WINFIELD RD, SAINT ALBANS, WV",38.423250,-81.854750,540790206091045.0
4,5506907200,WV,"2520 VALLEY DR, POINT PLEASANT, WV",38.866192,-82.124736,540539550004006.0
...,...,...,...,...,...,...
3360,7867607002,WV,"903 Grand Central Ave, VIENNA, WV",39.308604,-81.553172,541070105012030.0
3361,8194978504,WV,"218 D St, South Charleston, WV",38.365271,-81.693659,540390129001050.0
3362,9443798505,WV,"917 Deer Ridge Road, Davis, WV",39.052267,-79.410183,540939653002118.0
3363,9755898608,WV,"818 W Main St, Bridgeport, WV",39.282200,-80.266586,540330311002018.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
33,1092927109,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1076,9253917006,WV,"147 11TH AVE STE 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
1246,7284168605,WV,"5450 US Route 60, Huntington, WV",37.068304,-88.705767,NaN
1377,6167388303,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
1378,5082197000,WV,"2334 Route 52, KENOVA, WV",35.396151,-80.199428,NaN
2472,6046968304,WV,"2334 Route 52, Kenova, WV",35.396151,-80.199428,NaN
3164,5550167102,WV,"147 11TH AVE ste 100, SOUTH CHARLESTON, WV",40.748221,-74.007610,NaN
3186,1014389001,WV,"835 Oakdale Dr, Princeton, WV",33.776886,-84.338984,NaN


Convert to File

In [19]:
fips_WV = fips_merge_drop

In [20]:
fips_WV.to_csv("../../../data/state_data/geo/geo_fips/23/WV_fips_scraped.csv") 